# lab5

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

sns.set_style("darkgrid")
%matplotlib inline

In [5]:
df = pd.read_csv("Data.csv")

In [6]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [7]:
df.dropna(inplace=True)
df["Customer ID"] = df["Customer ID"].astype(int)

## Интересующие события и события наблюдения

в качестве событий будем использовать возвраты товаров

In [63]:
df['is_returned'] = df["Quantity"] < 0

In [64]:
median_price = df['Price'].median()
median_price

1.95

> событие наблюдения: цена выше медианной
> 
> интересующее событие: конкретный товар вернули

In [65]:
((df["is_returned"] == True) & (df['Price'] > median_price)).sum()

10890

## Полное совместное распределение

In [72]:
filtered_df = df[(df['Price'] > median_price)].copy()

In [73]:
filtered_df['is_returned'].sum()

10890

In [74]:
# Группировка данных по всем колонкам и подсчет частоты каждой уникальной комбинации
frequency_table = filtered_df.groupby(['is_returned', 'StockCode', 'Country']).size().reset_index(name='Frequency')

In [75]:
# Сумма всех частот для нормализации
total = np.sum(frequency_table['Frequency'])

# Вычисление вероятности для каждой комбинации
frequency_table['Probability'] = frequency_table['Frequency'] / total

In [76]:
assert np.isclose(frequency_table['Probability'].sum(), 1)

In [77]:
frequency_table

,is_returned,StockCode,Country,Frequency,Probability
0,False,10135,United Kingdom,6,0.000017
1,False,11001,United Kingdom,5,0.000014
2,False,15044A,Canada,1,0.000003
3,False,15044A,Channel Islands,1,0.000003
4,False,15044A,EIRE,3,0.000009
...,...,...,...,...,...
15649,True,POST,Spain,4,0.000011
15650,True,POST,Sweden,5,0.000014
15651,True,POST,Switzerland,3,0.000009
15652,True,POST,United Kingdom,93,0.000265


## Причины следствия

> возможно в стране United Kingdom возвращают товары чаще

In [80]:
# частная вероятность
frequency_table_part = df.groupby(['is_returned', 'Country']).size().reset_index(name='Frequency')
total = np.sum(frequency_table_part['Frequency'])
frequency_table_part['Probability'] = frequency_table_part['Frequency'] / total

In [79]:
frequency_table_part

,is_returned,Country,Frequency,Probability
0,False,Australia,1757,0.002398
1,False,Austria,845,0.001153
2,False,Bahrain,59,0.000081
3,False,Belgium,2842,0.003880
4,False,Brazil,94,0.000128
...,...,...,...,...
67,True,Switzerland,51,0.000070
68,True,USA,115,0.000157
69,True,United Arab Emirates,3,0.000004
70,True,United Kingdom,14817,0.020226


In [81]:
# условная вероятность
frequency_table_cond = filtered_df[filtered_df['Country'] == 'United Kingdom'].groupby(['is_returned', 'StockCode']).size().reset_index(name='Frequency')
total = np.sum(frequency_table_cond['Frequency'])
frequency_table_cond['Probability'] = frequency_table_cond['Frequency'] / total
frequency_table_cond

,is_returned,StockCode,Frequency,Probability
0,False,10135,6,0.000019
1,False,11001,5,0.000016
2,False,15044A,171,0.000544
3,False,15044B,136,0.000433
4,False,15044C,147,0.000468
...,...,...,...,...
4194,True,D,127,0.000404
4195,True,M,243,0.000773
4196,True,PADS,1,0.000003
4197,True,POST,93,0.000296


## Предсказание событий

In [120]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline 
from sklearn.compose import make_column_transformer, make_column_selector

In [132]:
xy = df[['Country', 'Price', 'is_returned']]
X = df[['Country', 'Price']].copy()
y = df['is_returned'].copy()

category_columns = ['Country']

X[category_columns] = X[category_columns].astype('category')

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=42)

In [150]:
ordinal_encoder = make_column_transformer(
    (
        OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
        make_column_selector(dtype_include="category"),
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
)

pipeline = make_pipeline(
    ordinal_encoder,
    HistGradientBoostingClassifier(
        categorical_features=[0],
        random_state=43,
    ),
)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f59dec871d0>)],
                                   verbose_feature_names_out=False)),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(categorical_features=[0],
                                                random_state=43))])

In [151]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, pipeline.predict(X_test))

0.976570093151762